# Vizualizacija neurona u kitty mreži



### Definicije i importovi

In [ ]:
pip install --quiet torch-lucent

In [ ]:
from lucent.optvis.transform import pad, jitter, random_rotate, random_scale
from lucent.optvis import render, param, transform, objectives

In [ ]:
import torch

from lucent.optvis import render, param, transform, objectives

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class KittyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0 = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(3, 9, 3)
        self.pool1 = nn.AvgPool2d(4, 4)
        
        self.conv1_bn = nn.BatchNorm2d(9)
        self.conv2 = nn.Conv2d(9, 16, 3)
        self.pool2 = nn.AvgPool2d(4, 4)
        
        self.conv2_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 25, 3)
        self.pool3 = nn.AvgPool2d(4, 4)
        
        self.conv3_bn = nn.BatchNorm2d(25)
        self.conv4 = nn.Conv2d(25, 36, 3)
        self.pool4 = nn.AvgPool2d(2 , 2)
        
        self.fc = nn.Linear(324, 4)

    def forward(self, x):
        x = self.bn0(x)
        x = self.conv1_bn(self.pool1(F.relu(self.conv1(x))))
        x = self.conv2_bn(self.pool2(F.relu(self.conv2(x))))
        x = self.conv3_bn(self.pool3(F.relu(self.conv3(x))))
        x = self.pool4(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc(x)
        return x
    
class LongcatNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(3, 9, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2_bn = nn.BatchNorm2d(9)
        self.conv2 = nn.Conv2d(9, 16, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 25, 3)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4_bn = nn.BatchNorm2d(25)
        self.conv4 = nn.Conv2d(25, 36, 3)
        self.pool4 = nn.MaxPool2d(2, 2)
        
        self.conv5_bn = nn.BatchNorm2d(36)
        self.conv5 = nn.Conv2d(36, 36, 3)
  
        self.conv6_bn = nn.BatchNorm2d(36)
        self.conv6 = nn.Conv2d(36, 49, 3)

        self.conv7_bn = nn.BatchNorm2d(49)
        self.conv7 = nn.Conv2d(49, 49, 3)
        
        self.conv8_bn = nn.BatchNorm2d(49)
        self.conv8 = nn.Conv2d(49, 49, 3)
        
        self.conv9_bn = nn.BatchNorm2d(49)
        self.conv9 = nn.Conv2d(49, 49, 3)
        self.pool9 = nn.MaxPool2d(2, 2)

        self.conv10_bn = nn.BatchNorm2d(49)
        self.conv10 = nn.Conv2d(49, 49, 3)
        self.pool10 = nn.MaxPool2d(2, 2)

        self.fc = nn.Linear(1764, 4)

    def forward(self, x):
        x = self.bn1(x)
        x = self.conv2_bn(self.pool1(F.relu(self.conv1(x))))
        x = self.conv3_bn(self.pool2(F.relu(self.conv2(x))))
        x = self.conv4_bn(self.pool3(F.relu(self.conv3(x))))
        x = self.conv5_bn(self.pool4(F.relu(self.conv4(x))))

        x = self.conv6_bn(F.relu(self.conv5(x)))  
        x = self.conv7_bn(F.relu(self.conv6(x)))
        x = self.conv8_bn(F.relu(self.conv7(x)))
        x = self.conv9_bn(F.relu(self.conv8(x)))
        
        
        x = self.conv10_bn(self.pool9(F.relu(self.conv9(x))))
        x = self.pool10(F.relu(self.conv10(x)))
        
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        x = self.fc(x)
        return x

In [ ]:
kitty = KittyNet()
longcat = LongcatNet()
inception = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)


kitty.load_state_dict(torch.load('saved_models/kitty/epoch_7_batch_5000.pth', map_location=device))
longcat.load_state_dict(torch.load('saved_models/longcat/epoch_7_batch_5000.pth', map_location=device))
inception.load_state_dict(torch.load('saved_models/inception/epoch_7_batch_5000.pth', map_location=device))

In [ ]:
kitty.to(device).eval()
longcat.to(device).eval()
inception.to(device).eval()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [ ]:
def lucent_show_layer(model, layer, grid_dim,
                      param_f=None, transforms=None,
                      optimizer=None, image_size=128):
    n_row = grid_dim
    n_col = grid_dim
    _, axs = plt.subplots(n_row, n_col, figsize=(19.55, 20))
    axs = axs.flatten()
    for ix, ax in zip(range(n_row*n_col), axs):
        img = render.render_vis(model, f"{layer}:{ix}", param_f=param_f,
                                transforms=transforms, progress=False, show_image=False)[0]
        img = np.reshape(img, (image_size, image_size, 3))
        ax.imshow(img)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.margins(x=0, y=0, tight=True)

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

## Različiti inputi

Prikazujemo razlličite inpute koji jako aktiviraju određeni neuron.

## Kitty primjeri

In [ ]:
batch_param_f = lambda: param.image(128, batch=4)

In [ ]:
obj = objectives.channel("conv1", 0) - 5e1 * objectives.diversity("conv1") # here we use a higher weight on the diversity term

_ = render.render_vis(kitty, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv2", 0) - 1e2 * objectives.diversity("conv2") # here we use a higher weight on the diversity term

_ = render.render_vis(kitty, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv3", 0) - 1e3 * objectives.diversity("conv3") # here we use a higher weight on the diversity term

_ = render.render_vis(kitty, obj, batch_param_f, show_inline=True)

### Longcat primjeri

In [ ]:
obj = objectives.channel("conv1", 0) - 1e2 * objectives.diversity("conv1") # here we use a higher weight on the diversity term

_ = render.render_vis(longcat, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv2", 2) - 1e3 * objectives.diversity("conv2") # here we use a higher weight on the diversity term

_ = render.render_vis(longcat, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv3", 2) - 1e3 * objectives.diversity("conv3") # here we use a higher weight on the diversity term

_ = render.render_vis(longcat, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv4", 2) - 1e3 * objectives.diversity("conv4") # here we use a higher weight on the diversity term

_ = render.render_vis(longcat, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv5", 0) - 1e3 * objectives.diversity("conv5") # here we use a higher weight on the diversity term

_ = render.render_vis(longcat, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv6", 0) - 1e3 * objectives.diversity("conv6") # here we use a higher weight on the diversity term

_ = render.render_vis(longcat, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv7", 1) - 1e3 * objectives.diversity("conv7") # here we use a higher weight on the diversity term

_ = render.render_vis(longcat, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv8", 1) - 1e3 * objectives.diversity("conv8") # here we use a higher weight on the diversity term

_ = render.render_vis(longcat, obj, batch_param_f, show_inline=True)

In [ ]:
obj = objectives.channel("conv9", 1) - 5e2 * objectives.diversity("conv9") # here we use a higher weight on the diversity term

_ = render.render_vis(longcat, obj, batch_param_f, show_inline=True)

### Inception

In [ ]:
batch_param_f = lambda: param.image(256, fft=True, batch=4)

obj = objectives.channel("inception4a", 97) - 1e2 * objectives.diversity("inception4a") # here we use a higher weight on the diversity term

_ = render.render_vis(inception, obj, batch_param_f, show_inline=True)

In [ ]:
batch_param_f = lambda: param.image(256, fft=True, batch=4)

obj = objectives.channel("inception4a", 98) - 1e2 * objectives.diversity("inception4a") # here we use a higher weight on the diversity term

_ = render.render_vis(inception, obj, batch_param_f, show_inline=True)

In [ ]:
batch_param_f = lambda: param.image(256, fft=True, batch=4)

obj = objectives.channel("inception4b", 98) - 1e2 * objectives.diversity("inception4a") # here we use a higher weight on the diversity term

_ = render.render_vis(inception, obj, batch_param_f, show_inline=True)

In [ ]:
batch_param_f = lambda: param.image(256, fft=True, decorrelate=True, batch=4)

obj = objectives.channel("inception5b", 56) + objectives.channel("inception5b", 55) - 1e5 * objectives.diversity("inception5b") # here we use a higher weight on the diversity term

_ = render.render_vis(inception, obj, batch_param_f, show_inline=True)